輸出cookie尾端會有一個", 請自行用excel replace公式去除(不能用取代)

In [1]:
import pandas as pd
import numpy as np
from urllib.parse import unquote
from urllib.parse import quote
import xlsxwriter

In [2]:
# 轉碼器
def urlencode(text):
    if text!= None:
        return unquote(text, 'utf-8')
    else:
        return ""

def textencode(text):
    if text!= None:
        return quote(text)
    else:
        return ""

def listencode(text_list):
    answer = []
    for i in text_list:
        answer.append(textencode(i))
    return answer


In [3]:
# 設定顯示為數字(不要變成科學記號)
np.set_printoptions(suppress=True, formatter=dict(float=lambda x:"%19.0f" % x)) 
pd.set_option('display.float_format',lambda x : '%.0f' % x)

In [4]:
# 輸入兩檔案，分別為"頁面+cookie"&"事件+頁面+cookie"
rawdata_event_path = "(MG-事件頁面)(2020-03-19-2020-06-18)(0).xlsx"
rawdata_page_path = "(MG-頁面cookie)(2020-03-19-2020-06-18)(0).xlsx"

In [5]:
# 判斷式 : 
'''
可在此調整
"":{"type": "isin/contains/others", "check_column": [], "check_items": [], "excluded_items" : ["偵錯用神龕"]},
isin 代表完全相符, contains 代表包含, check_column設定欄位, check_items設定關鍵字, excluded_items設定排除字詞
'''
sort_dict = {
    # "":{"type": "isin/contains/others", "check_column": [], "check_items": [], "excluded_items" : ["偵錯用神龕"]},
    # "全站訪客":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["mamiguide", "Q"], "excluded_items" : []},
    "首頁":{"type": "isin", "check_column": ["頁面路徑"], "check_items": ["/"], "excluded_items" : ["偵錯用神龕"]},
    "懷孕前":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["生產準備", "生產包準備", "懷孕Q&A", "政府津貼", "產前", "孕前", "懷孕初期", "懷孕前三個月"], "excluded_items" : ["偵錯用神龕"]},
    "懷孕中":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["初期", "前三個月", "受精", "驗孕", "受孕", "害喜", "孕吐", "孕初", "媽媽手冊", "胃口不佳", "噁心", "懷孕", "生產", "孕婦", "媽媽教室", "新生兒", "親子教育", "待產", "哺乳", "親子教養"], "excluded_items" : ["生產準備", "生產包準備", "懷孕Q&A", "政府津貼", "產前", "孕前", "懷孕初期", "懷孕前三個月", "產後大小事", "月子中心", "哺乳專區", "產後", "寶寶照護", "新生兒", "親子", "哺乳"]},
    "懷孕後":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["哺乳", "月子中心", "哺乳專區", "產後", "寶寶照護", "新生兒", "親子", "哺乳"], "excluded_items" : ["偵錯用神龕"]},
    "孕早期":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["初期", "前三個月", "受精", "驗孕", "受孕", "害喜", "孕吐", "孕初", "媽媽手冊", "胃口不佳", "噁心"], "excluded_items" : ["偵錯用神龕"]},
    "孕中期":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["懷孕", "生產", "孕婦", "孕期"], "excluded_items" : ["初期", "前三個月", "受精", "驗孕", "受孕", "害喜", "孕吐", "孕初", "媽媽手冊", "胃口不佳", "噁心", "媽媽教室", "新生兒", "親子教育", "待產"]},
    "孕晚期":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["媽媽教室", "新生兒", "親子教育", "待產", "哺乳", "親子教養"], "excluded_items" : ["偵錯用神龕"]},
    "全文章專區":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["孕媽咪指南"], "excluded_items" : ["偵錯用神龕"]}, 
    "懷孕大小事":{"type": "isin", "check_column": ["事件標籤", "頁面URL"], "check_items": ["懷孕大小事", "懷孕注意事項", "懷孕禁忌", "懷孕食譜/飲食"], "excluded_items" : ["偵錯用神龕"]}, 
    "懷孕好康":{"type": "isin", "check_column": ["事件標籤", "頁面URL"], "check_items": ["懷孕好康", "2020媽媽手冊換贈品", "全省媽媽教室", "月子中心晚鳥優惠"], "excluded_items" : ["偵錯用神龕"]}, 
    "生產準備":{"type": "isin", "check_column": ["事件標籤", "頁面URL"], "check_items": ["生產準備", "待產包準備", "懷孕Q&A"], "excluded_items" : ["偵錯用神龕"]}, 
    "政府津貼":{"type": "isin", "check_column": ["事件標籤", "頁面URL"], "check_items": ["政府津貼", "產前健康補助"], "excluded_items" : ["偵錯用神龕"]}, 
    "產後大小事":{"type": "isin", "check_column": ["事件標籤", "頁面URL"], "check_items": ["產後大小事", "月子中心", "哺乳專區", "產後保養"], "excluded_items" : ["偵錯用神龕"]}, 
    "孕前準備":{"type": "isin", "check_column": ["事件標籤", "頁面URL"], "check_items": ["孕前準備", "孕前檢查", "孕前注意事項"], "excluded_items" : ["偵錯用神龕"]}, 
    "寶寶照護":{"type": "isin", "check_column": ["事件標籤", "頁面URL"], "check_items": ["寶寶照護", "新生兒照護"], "excluded_items" : ["偵錯用神龕"]}, 
    "親子教養":{"type": "isin", "check_column": ["事件標籤", "頁面URL"], "check_items": ["親子教養", "親子教育"], "excluded_items" : ["偵錯用神龕"]}, 
    "補品照護":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["西餐", "中餐", "西藥", "中藥", "葉酸", "薑母鴨", "補充營養", "補品", "人參", "藥膳", "月子餐等"], "excluded_items" : ["偵錯用神龕"]}, 
    "懷孕資訊":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["懷孕三餐食譜", "懷孕飲食建議", "懷孕三餐怎麼吃"], "excluded_items" : ["偵錯用神龕"]}, 
    "相關症狀":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["假性月經", "妊娠", "孕吐", "害喜", "漲奶"], "excluded_items" : ["偵錯用神龕"]}, 
    "寶寶相關":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["寶寶照護", "寶寶按摩", "寶寶游泳"], "excluded_items" : ["偵錯用神龕"]}, 
    "月子中心":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["月子中心", "產後護理之家", "價格", "設備", "環境", "實住心得", "評鑑", "廚房", "高CP值", "五星級", "醫護比", "中醫師", "頂級", "參觀心得", "實住", "評鑑", "便宜", "親餵"], "excluded_items" : ["偵錯用神龕"]}, 
    "媽媽教室":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["媽媽教室", "媽媽手冊"], "excluded_items" : ["偵錯用神龕"]}, 
    "台北":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["台北", "臺北"], "excluded_items" : ["偵錯用神龕"]},
    "新北":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["新北"], "excluded_items" : ["偵錯用神龕"]},
    "桃園":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["桃園"], "excluded_items" : ["偵錯用神龕"]},
    "台中":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["台中", "臺中"], "excluded_items" : ["偵錯用神龕"]},
    "台南":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["台南", "臺南"], "excluded_items" : ["偵錯用神龕"]},
    "高雄":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["高雄"], "excluded_items" : ["偵錯用神龕"]},
    "高價":{"type": "contains", "check_column": ["事件標籤", "頁面URL"], "check_items": ["按摩", "游泳", "攝影", "瑜珈", "SPA", "洗頭", "美甲", "人參","度假"], "excluded_items" : ["偵錯用神龕"]},
    
}

In [6]:
# 區分欄位資料網址還是文字
# 若要使用頁面路徑完全相符請將該類別名稱放入urlpath_exactly_match
action_list = ["頁面標題", "事件分類", "事件動作", "事件標籤"]
url_list = ["頁面URL", "頁面路徑"]
urlpath_exactly_match = ["首頁"]

In [7]:
df_event = pd.read_excel(rawdata_event_path)
df_page = pd.read_excel(rawdata_page_path)

In [8]:
df_event.shape, df_page.shape

((1048575, 12), (211681, 10))

In [9]:
# dropna cookie
df_event = df_event.dropna(subset = ["admckid"])
df_page = df_page.dropna(subset = ["admckid"])

In [10]:
df_event.head(1)

,admckid,頁面路徑,頁面URL,頁面標題,事件分類,事件動作,事件標籤,事件價值,事件總數,唯一身份訪問者,事件總價值,平均事件價值
5818,1512300932011351552,/%E5%AD%95%E5%AA%BD%E5%92%AA%E6%8C%87%E5%8D%97...,https://mamiguide.com/%E5%AD%95%E5%AA%BD%E5%92...,不知道懷孕吃了薑母鴨會怎樣？1張圖了解懷孕吃薑母鴨會面臨的4大問題 | MamiGuide孕...,文章,文章分類,懷孕大小事,0,1,1,0,0


In [11]:
df_page.head(1)

,admckid,頁面路徑,頁面URL,頁面標題,唯一身份訪問者,瀏覽量,唯一身份瀏覽量,跳出率,退出率,平均頁面停留時間
748,1601061359001688320,/%E5%8F%B0%E5%8C%97%E5%B8%82,https://mamiguide.com/%E5%8F%B0%E5%8C%97%E5%B8%82,【台北市月子中心推薦】59間超人氣台北市產後護理之家設備和評價-MamiGuide,1,1,1,1,1,94


In [12]:
'''
# url僅用於全文章，欄位:頁面URL
check_url = ["孕媽咪指南"]
# title，欄位:頁面標題
check_title= []
# category文章分類，欄位:事件標籤
check_category = ["懷孕大小事", "懷孕注意事項", "懷孕禁忌", "懷孕食譜/飲食", "懷孕好康", "2020媽媽手冊換贈品", "全省媽媽教室", "月子中心晚鳥優惠", "生產準備", "待產包準備", "懷孕Q&A", "政府津貼", "產前健康補助", "產後大小事", "月子中心", "哺乳專區", "產後保養", "孕前準備", "孕前檢查", "孕前注意事項", "寶寶照護", "新生兒照護", "親子教養", "親子教育"]
# label文章標籤，事件動作 = 文章標籤，欄位:事件標籤，頁面URL
check_label = ["西藥", "中藥", "補品", "人參", "藥膳", "月子餐", "懷孕三餐食譜", "懷孕飲食建議", "假性月經", "妊娠", "孕吐", "害喜", "漲奶", "寶寶照護", "寶寶按摩", "寶寶游泳", "月子中心", "產後護理之家", "價格", "設備", "環境", "實住心得", "評鑑", "媽媽教室", "媽媽手冊", "初期", "前三個月", "受精", "驗孕", "受孕", "孕初", "胃口不佳", "噁心", "新生兒", "親子教育", "待產", "懷孕", "孕婦", "孕期"]
# region地區，頁面動作=地區搜尋月子中心、搜尋月子中心-熱門縣市、找月子中心、月子中心地區、熱搜月子中心
check_region = ["基隆", "嘉義", "台北", "嘉義", "新北", "台南", "桃園", "高雄", "新竹", "屏東", "新竹", "台東", "苗栗", "花蓮", "台中", "宜蘭", "彰化", "澎湖", "南投", "金門", "雲林", "連江"]
# price事件動作:勾選價格下限(價值)、月子中心價格下限(標籤)、房型價格(標籤)
check_price = [0, 5500, 6500, 8000]
# all checking-items
check_list = [check_url, check_title, check_category, check_label, check_region, check_price]
'''

'\n# url僅用於全文章，欄位:頁面URL\ncheck_url = ["孕媽咪指南"]\n# title，欄位:頁面標題\ncheck_title= []\n# category文章分類，欄位:事件標籤\ncheck_category = ["懷孕大小事", "懷孕注意事項", "懷孕禁忌", "懷孕食譜/飲食", "懷孕好康", "2020媽媽手冊換贈品", "全省媽媽教室", "月子中心晚鳥優惠", "生產準備", "待產包準備", "懷孕Q&A", "政府津貼", "產前健康補助", "產後大小事", "月子中心", "哺乳專區", "產後保養", "孕前準備", "孕前檢查", "孕前注意事項", "寶寶照護", "新生兒照護", "親子教養", "親子教育"]\n# label文章標籤，事件動作 = 文章標籤，欄位:事件標籤，頁面URL\ncheck_label = ["西藥", "中藥", "補品", "人參", "藥膳", "月子餐", "懷孕三餐食譜", "懷孕飲食建議", "假性月經", "妊娠", "孕吐", "害喜", "漲奶", "寶寶照護", "寶寶按摩", "寶寶游泳", "月子中心", "產後護理之家", "價格", "設備", "環境", "實住心得", "評鑑", "媽媽教室", "媽媽手冊", "初期", "前三個月", "受精", "驗孕", "受孕", "孕初", "胃口不佳", "噁心", "新生兒", "親子教育", "待產", "懷孕", "孕婦", "孕期"]\n# region地區，頁面動作=地區搜尋月子中心、搜尋月子中心-熱門縣市、找月子中心、月子中心地區、熱搜月子中心\ncheck_region = ["基隆", "嘉義", "台北", "嘉義", "新北", "台南", "桃園", "高雄", "新竹", "屏東", "新竹", "台東", "苗栗", "花蓮", "台中", "宜蘭", "彰化", "澎湖", "南投", "金門", "雲林", "連江"]\n# price事件動作:勾選價格下限(價值)、月子中心價格下限(標籤)、房型價格(標籤)\ncheck_price = [0, 5500, 6500, 8000]\n# all checking-items\ncheck_list = [check

In [12]:
np.unique(df_event[((~df_event["事件標籤"].isna()) & (df_event["事件標籤"].str.contains("假性月經|妊娠"))) | ((~df_event["事件標籤"].isna()) & (df_event["事件標籤"].str.contains("孕吐|月子餐")))]["admckid"])

array([1601011835091900416, 1603062142031718144, 1604131019291774720, ...,
       2006181843571521280, 2006182028371298560, 2006182311251100160])

In [13]:
def cookie_to_str(cookie):
    temp = [int(x) for x in cookie]
    return([(str(x) +'"')for x in temp])

# 篩選

In [14]:
cookie_list = [[0]]
column_name = ["全站訪客"]

In [15]:
cookie_list[0].extend(cookie_to_str(list(set(df_page["admckid"]))))
cookie_list[0][0] = len(cookie_list[0]) - 1

In [16]:
'''
for k,v in sort_dict.items():
    column_name.append(k)
    temp_list = []
    print(k,v)
    if v["type"] == "isin":
        temp_list.extend(list(set(df_event[((~df_event[v["check_column"][0]].isna()) & (df_event[v["check_column"][0]].isin(v["check_items"])) & (~df_event[v["check_column"][0]].isin(v["excluded_items"])))]["admckid"])))
        
    else:
        temp_list.extend(list(df_event[((~df_event[v["check_column"][0]].isna()) & (df_event[v["check_column"][0]].str.contains("|".join(v["check_items"]), na=False)) & (~df_event[v["check_column"][0]].str.contains("|".join(v["excluded_items"]), na=False)))]["admckid"]))
    
    print(len(set(temp_list)))
    
    temp_list.extend(list(set(df_page[(~df_page[v["check_column"][1]].isna()) & (df_page[v["check_column"][1]].str.contains("|".join(listencode(v["check_items"])), na=False)) & (~df_page[v["check_column"][1]].str.contains("|".join(listencode(v["excluded_items"])), na=False))]["admckid"])))

    print(len(set(temp_list)))
    
    temp = cookie_to_str(list(set((temp_list))))
    cookie_list.append(temp)
'''

'\nfor k,v in sort_dict.items():\n    column_name.append(k)\n    temp_list = []\n    print(k,v)\n    if v["type"] == "isin":\n        temp_list.extend(list(set(df_event[((~df_event[v["check_column"][0]].isna()) & (df_event[v["check_column"][0]].isin(v["check_items"])) & (~df_event[v["check_column"][0]].isin(v["excluded_items"])))]["admckid"])))\n        \n    else:\n        temp_list.extend(list(df_event[((~df_event[v["check_column"][0]].isna()) & (df_event[v["check_column"][0]].str.contains("|".join(v["check_items"]), na=False)) & (~df_event[v["check_column"][0]].str.contains("|".join(v["excluded_items"]), na=False)))]["admckid"]))\n    \n    print(len(set(temp_list)))\n    \n    temp_list.extend(list(set(df_page[(~df_page[v["check_column"][1]].isna()) & (df_page[v["check_column"][1]].str.contains("|".join(listencode(v["check_items"])), na=False)) & (~df_page[v["check_column"][1]].str.contains("|".join(listencode(v["excluded_items"])), na=False))]["admckid"])))\n\n    print(len(set(te

In [16]:
for k,v in sort_dict.items():
    column_name.append(k)
    temp_list = [0]
    print(k,v)
    for check_column in v["check_column"]:
        '''
        if "price" in v:
            try:
                df_price = df_price.append(df_event[(~df_event[check_column].isna()) & (df_event[check_column].isin(v["check_items"]))])
            except:
                df_price = df_event[(~df_event[check_column].isna()) & (df_event[check_column].isin(v["check_items"]))]  
        '''    
        if check_column in action_list:
            if v["type"] == "isin":
                temp_list.extend(list(set(df_event[((~df_event[check_column].isna()) & (df_event[check_column].isin(v["check_items"])) & (~df_event[check_column].isin(v["excluded_items"])))]["admckid"])))
            elif v["type"] == "contains":
                temp_list.extend(list(df_event[((~df_event[check_column].isna()) & (df_event[check_column].str.contains("|".join(v["check_items"]), na=False)) & (~df_event[check_column].str.contains("|".join(v["excluded_items"]), na=False)))]["admckid"]))
        elif check_column in url_list:
            if k in urlpath_exactly_match:
                temp_list.extend(list(set(df_page[df_page[check_column].isin(v["check_items"])]["admckid"])))
            else:
                temp_list.extend(list(set(df_page[(~df_page[check_column].isna()) & (df_page[check_column].str.contains("|".join(listencode(v["check_items"])), na=False)) & (~df_page[check_column].str.contains("|".join(listencode(v["excluded_items"])), na=False))]["admckid"])))

        print(len(set(temp_list))-1)
    
    temp = cookie_to_str(list(set((temp_list))))
    temp[0] = len(temp)
    cookie_list.append(temp)

首頁 {'type': 'isin', 'check_column': ['頁面路徑'], 'check_items': ['/'], 'excluded_items': ['偵錯用神龕']}
4475
懷孕前 {'type': 'contains', 'check_column': ['事件標籤', '頁面URL'], 'check_items': ['生產準備', '生產包準備', '懷孕Q&A', '政府津貼', '產前', '孕前', '懷孕初期', '懷孕前三個月'], 'excluded_items': ['偵錯用神龕']}
49988
50724
懷孕中 {'type': 'contains', 'check_column': ['事件標籤', '頁面URL'], 'check_items': ['初期', '前三個月', '受精', '驗孕', '受孕', '害喜', '孕吐', '孕初', '媽媽手冊', '胃口不佳', '噁心', '懷孕', '生產', '孕婦', '媽媽教室', '新生兒', '親子教育', '待產', '哺乳', '親子教養'], 'excluded_items': ['生產準備', '生產包準備', '懷孕Q&A', '政府津貼', '產前', '孕前', '懷孕初期', '懷孕前三個月', '產後大小事', '月子中心', '哺乳專區', '產後', '寶寶照護', '新生兒', '親子', '哺乳']}
65801
66451
懷孕後 {'type': 'contains', 'check_column': ['事件標籤', '頁面URL'], 'check_items': ['哺乳', '月子中心', '哺乳專區', '產後', '寶寶照護', '新生兒', '親子', '哺乳'], 'excluded_items': ['偵錯用神龕']}
38543
40117
孕早期 {'type': 'contains', 'check_column': ['事件標籤', '頁面URL'], 'check_items': ['初期', '前三個月', '受精', '驗孕', '受孕', '害喜', '孕吐', '孕初', '媽媽手冊', '胃口不佳', '噁心'], 'excluded_items': ['偵錯用神龕']}
5

In [18]:
# 孕中期、懷孕中去重
cookie_list[6] = list(set(cookie_list[6]) - set(cookie_list[5]) - set(cookie_list[7]))
cookie_list[3] = list((set(cookie_list[5]) | set(cookie_list[6]) | set(cookie_list[7])) - set(cookie_list[2]) - set(cookie_list[4]))

In [19]:
df_final = pd.DataFrame(cookie_list[0], columns = [column_name[0]])
print(0, column_name[0], len(cookie_list[0])-1)
for col in range(1, len(column_name)):
    print(col, column_name[col], len(cookie_list[col])-1)
    df_temp = pd.DataFrame({column_name[col]: cookie_list[col]})
    df_final = pd.concat([df_final,df_temp], axis=1)

0 全站訪客 98623
1 首頁 4475
2 懷孕前 50724
3 懷孕中 4517
4 懷孕後 40117
5 孕早期 53898
6 孕中期 9782
7 孕晚期 8872
8 全文章專區 71280
9 懷孕大小事 38855
10 懷孕好康 3459
11 生產準備 1611
12 政府津貼 1957
13 產後大小事 34547
14 孕前準備 18620
15 寶寶照護 2783
16 親子教養 113
17 補品照護 28274
18 懷孕資訊 2931
19 相關症狀 21741
20 寶寶相關 3311
21 月子中心 44424
22 媽媽教室 4629
23 台北 6999
24 新北 6397
25 桃園 3742
26 台中 5204
27 台南 3655
28 高雄 3606
29 高價 10803


In [20]:
df_final

,全站訪客,首頁,懷孕前,懷孕中,懷孕後,孕早期,孕中期,孕晚期,全文章專區,懷孕大小事,...,寶寶相關,月子中心,媽媽教室,台北,新北,桃園,台中,台南,高雄,高價
0,98623,4476,50725,"2003090949151635968""",40118,53899,"1803220958471128064""",8873,71281,38856,...,3312,44425,4630,7000,6398,3743,5205,3656,3607,10804
1,"1712092123431698432""","1805191854051786752""","1802240026461601792""","2002211043341683712""","1704170854291734528""","1612191803081883648""","1910311714361960960""","1707121912271863808""","1612191803081883648""","1707121912271863808""",...,"1709101313021304832""","1704170854291734528""","1612092133531443200""","1704232243461849088""","1907280121161383936""","1802270134511861760""","1704170854291734528""","1709151212471566336""","1812161756391342080""","1804051137141276672"""
2,"1802270134511861760""","1812100341351964672""","1805181948331687936""","2001210750431564544""","1802270134511861760""","1802240026461601792""","1911120047431998720""","1904011240581562368""","1805131430161547264""","1901211938311766016""",...,"1805181808581459968""","1802270134511861760""","1704252304421142528""","1802260045211893760""","2005221204361478144""","1803310851521789952""","1803182209181155328""","1811161841021796352""","1905051001001648128""","1911120718191656960"""
3,"1809302324481359872""","2001061121061412864""","1809291946171301888""","2005052036581291008""","1805231530421780480""","1809302324481359872""","2003241907571670528""","1911241529391906816""","1810132036451696640""","1704120013571358720""",...,"1809142203181187072""","1805231530421780480""","1806201313471365120""","1812171522351136768""","1906262159401484288""","1901220949431255040""","1802200134581911552""","1902251435571609600""","1905211517541130240""","1801101046331768832"""
4,"2003052006551846912""","1810220844321103872""","1901231641171722240""","1812121739051177472""","1809290203001520128""","1805181948331687936""","1911271623211744768""","1907141457301536768""","2001192210161205248""","1908301425331404800""",...,"1907271015141523456""","1809290203001520128""","1809302324481359872""","1811181849171984384""","2004030851231612928""","1907032354501230592""","1711042006121807872""","1911041521561690112""","1904081352131936256""","1901101244031795200"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98619,"2005172300021473280""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98620,"1901151551531597312""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98621,"1811210927571811840""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98622,"1906222135311416832""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df_final.to_excel("cookie_pack.xlsx")

In [22]:
df_final.to_csv("cookie_pack.csv", encoding="utf-8-sig")

In [23]:
df_final.to_csv("cookie_pack.txt")

# 測試區

In [91]:
df_event[df_event["頁面標題"].str.contains('|'.join(check_label))]

,admckid,頁面路徑,頁面URL,頁面標題,事件分類,事件動作,事件標籤,事件價值,事件總數,唯一身份訪問者,事件總價值,平均事件價值
3737,1.512301e+18,/%E5%AD%95%E5%AA%BD%E5%92%AA%E6%8C%87%E5%8D%97...,https://mamiguide.com/%E5%AD%95%E5%AA%BD%E5%92...,懷孕初期吃什麽好？４大懷孕初期飲食建議輕鬆吃出好營養！ | MamiGuide孕媽咪指南,文章,文章標籤,人參,0,3,1,0,0
3738,1.512301e+18,/%E5%AD%95%E5%AA%BD%E5%92%AA%E6%8C%87%E5%8D%97...,https://mamiguide.com/%E5%AD%95%E5%AA%BD%E5%92...,懷孕初期吃什麽好？４大懷孕初期飲食建議輕鬆吃出好營養！ | MamiGuide孕媽咪指南,文章,文章標籤,早產,0,3,1,0,0
3739,1.512301e+18,/%E5%AD%95%E5%AA%BD%E5%92%AA%E6%8C%87%E5%8D%97...,https://mamiguide.com/%E5%AD%95%E5%AA%BD%E5%92...,懷孕初期吃什麽好？４大懷孕初期飲食建議輕鬆吃出好營養！ | MamiGuide孕媽咪指南,文章,文章標籤,貧血,0,3,1,0,0
3740,1.512301e+18,/%E6%9C%88%E5%AD%90%E4%B8%AD%E5%BF%83/%E5%8F%B...,https://mamiguide.com/%E6%9C%88%E5%AD%90%E4%B8...,時尚產後護理之家-月子中心評價及設備，預約參觀,月子中心,月子中心價格,6200,0,1,1,0,0
3741,1.512301e+18,/%E6%9C%88%E5%AD%90%E4%B8%AD%E5%BF%83/%E5%8F%B...,https://mamiguide.com/%E6%9C%88%E5%AD%90%E4%B8...,時尚產後護理之家-月子中心評價及設備，預約參觀,月子中心,月子中心地區,台北市_松山區,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
582508,2.004262e+18,/%E5%AD%95%E5%AA%BD%E5%92%AA%E6%8C%87%E5%8D%97...,https://mamiguide.com/%E5%AD%95%E5%AA%BD%E5%92...,什麼時候驗孕最準？1張圖告訴你！ | MamiGuide孕媽咪指南,文章,文章標籤,推薦,0,1,1,0,0
582509,2.004262e+18,/%E5%AD%95%E5%AA%BD%E5%92%AA%E6%8C%87%E5%8D%97...,https://mamiguide.com/%E5%AD%95%E5%AA%BD%E5%92...,什麼時候驗孕最準？1張圖告訴你！ | MamiGuide孕媽咪指南,文章,文章標籤,驗孕,0,1,1,0,0
582510,2.004262e+18,/%E5%AD%95%E5%AA%BD%E5%92%AA%E6%8C%87%E5%8D%97...,https://mamiguide.com/%E5%AD%95%E5%AA%BD%E5%92...,懷孕初期假性月經要如何辨別？3種跡象輕鬆判斷 | MamiGuide孕媽咪指南,文章,文章標籤,假性月經,0,1,1,0,0
582511,2.004262e+18,/%E5%AD%95%E5%AA%BD%E5%92%AA%E6%8C%87%E5%8D%97...,https://mamiguide.com/%E5%AD%95%E5%AA%BD%E5%92...,懷孕初期假性月經要如何辨別？3種跡象輕鬆判斷 | MamiGuide孕媽咪指南,文章,文章標籤,懷孕前三個月,0,1,1,0,0


In [48]:
np.array(cookie_list[0]).shape

(35349,)

In [49]:
df_final

,全文章專區,懷孕大小事,懷孕好康,生產準備,政府津貼,產後大小事,孕前準備,寶寶照護,親子教養,補品照護,懷孕資訊,相關症狀,寶寶相關,月子中心,媽媽教室
0,1512300932011351552,1512300932011351552,1610120748311858944,1606161502081797632,1605281736021855232,1512300932011351552,1601161715221780480,1605171507011345920,1706150919061916416,1512300932011351552,1602012029171102720,1601011835091900416,1602101851581866496,1512300932011351552,1609221915331644160
1,1512312200441580032,1512312200441580032,1611060559161399296,1610111024331681792,1612112032381420032,1601182341371447808,1607132246411778304,1609151157001633792,1707141045051995648,1605171507011345920,1603062142031718144,1604031949041277440,1605171507011345920,1601181601511718912,1610120748311858944
2,1601011835091900416,1601011835091900416,1701022147041435136,1611041437131864320,1702141929131303424,1602012029171102720,1607200950471623680,1609221915331644160,1708301919181910016,1606042254471703552,1611130918181473280,1605210145431463936,1606292332491126272,1601182341371447808,1610152044391602688
3,1601161715221780480,1602012029171102720,1702040052311699968,1611081256101202944,1702181200181555456,1602101851581866496,1607261905371412736,1610152044391602688,1803011310371811584,1607291045061852416,1611191639231852544,1605232346241277440,1609020059591396864,1602012029171102720,1611060559161399296
4,1602012029171102720,1602090401441780736,1702142244561340416,1612042215571843072,1704230010331714816,1603062142031718144,1608111916001821184,1611271942081920512,1804110119121896960,1608021921091451904,1612202009491354112,1606021433181449216,1609151157001633792,1602101851581866496,1611271942081920512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35344,2004262247091420160,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
35345,2004262312311252736,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
35346,2004262321371220736,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
35347,2004262325011772160,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [25]:
temp_df[temp_df["頁面路徑"].isin(["/"])]

,admckid,頁面路徑,頁面URL,頁面標題,事件分類,事件動作,事件標籤,事件價值,事件總數,唯一身份訪問者,事件總價值,平均事件價值
2858,1702142244561340416,/,https://mamiguide.com/,產後月子中心顧問 五星級月子餐點 (推薦人氣平台) | MamiGuide孕媽咪指南,Header,孕媽咪指南,2020媽媽手冊換贈品,0,1,1,0,0
38391,1907011636411804160,/,https://mamiguide.com/,產後月子中心顧問 五星級月子餐點 (推薦人氣平台) | MamiGuide孕媽咪指南,首頁,專家顧問團隊,郭安妮,0,1,1,0,0
44061,1908191301191123968,/,https://mamiguide.com/,產後月子中心顧問 五星級月子餐點 (推薦人氣平台) | MamiGuide孕媽咪指南,Header,找月子中心,台北其他區域,0,5,1,0,0
44387,1908211149331732736,/,https://mamiguide.com/,產後月子中心顧問 五星級月子餐點 (推薦人氣平台) | MamiGuide孕媽咪指南,Header,孕媽咪指南,2020媽媽手冊換贈品,0,1,1,0,0
60941,2001091432031152384,/,https://mamiguide.com/,產後月子中心顧問 五星級月子餐點 (推薦人氣平台) | MamiGuide孕媽咪指南,Header,孕媽咪指南,2020媽媽手冊換贈品,0,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
501878,1912161417451465728,/,https://mamiguide.com/,產後月子中心顧問 五星級月子餐點 (推薦人氣平台) | MamiGuide孕媽咪指南,Header,孕媽咪指南,2020媽媽手冊換贈品,0,1,1,0,0
504613,2001021502541179392,/,https://mamiguide.com/,產後月子中心顧問 五星級月子餐點 (推薦人氣平台) | MamiGuide孕媽咪指南,首頁,精選文章懷孕專題,憑媽媽手冊，免費索取孕媽咪好禮,0,1,1,0,0
505360,2001070735261110016,/,https://mamiguide.com/,產後月子中心顧問 五星級月子餐點 (推薦人氣平台) | MamiGuide孕媽咪指南,Header,孕媽咪指南,2019媽媽手冊換贈品,0,1,1,0,0
514097,2002241333231375872,/,https://mamiguide.com/,產後月子中心顧問 五星級月子餐點 (推薦人氣平台) | MamiGuide孕媽咪指南,Header,孕媽咪指南,2020媽媽手冊換贈品,0,1,1,0,0


In [23]:
middle = list(set(cookie_list[6]) - set(cookie_list[5]) - set(cookie_list[7]))
len(middle), len(cookie_list[5]), len(cookie_list[6]), len(cookie_list[7])

9783

In [24]:
len(middle), len(cookie_list[5]), len(cookie_list[6]), len(cookie_list[7])

(9783, 53899, 66049, 8873)

In [26]:
middle2 = list((set(cookie_list[5]) | set(cookie_list[6]) | set(cookie_list[7])) - set(cookie_list[2]) - set(cookie_list[4]))
len(middle2)

4518

In [28]:
with open("result.txt", "w") as result:
    for i in middle2:
        result.write("%s\n"%i)